## Research Assistant Test 
### DECTI Unit - World Bank
**Jairo Enrique Galvis Avella**

August - 2024

### Context

The ForeignNames_2019_2020.csv and ForeignNames_2021.csv are datasets of commercial foreign firm names and their location countries. ForeignNames_2019_2020.csv contains firm names from 2019-2020. ForeignNames_2021.csv contains firm names from 2021.
Data entry has been conducted manually by multiple people with no quality control, and as a result, the same company may have been recorded under different names, for instance: “Teaboard Ltd.” and “Teaboard Limited”. 


In [ ]:
#Python version 3.11.5

# To run this notebook you need to install the follwoing packages:
# pip install pandas
# pip install fuzzywuzzy
# pip install tqdm
# pip install scikit-learn
# pip install ipython


### Part 1

Use ForeignNames_2019_2020.csv for the following tasks:

+ Clean the firm names and assign a unique identifier (ID) to each firm (include it in a new variable “cleaned_ID”).  Give a corresponding unique cleaned name to all the names (include it in a new variable called “cleaned_name”). For example, if Teaboard Limited, Teaboard LTd, Teaboar Ld all get assigned the same name, choose the best name to give to this group, lets say “Teaboard Limited”.

+ Please do this cleaning of firm names country by country (countries are identified by the ISO3 codes included in the file ForeignNames_2019_2020.csv and their mapping to country names is given in the separate file Country_Name_ISO3.csv we also sent you) as stopwords (words you discount in matching) may vary from country to country and also because it minimizes error. 

+ For all firm ID assignments, please follow a convention such that the country iso3 is attached in front of the numeric ID to identify the country of the firm. 

In [24]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm
from IPython.display import display

# Set the working directory
os.chdir('/Users/jairogalvis/Downloads/PythonRStataTest2024/PythonTest/')

# Load the datasets
foreign_names = pd.read_csv('ForeignNames_2019_2020.csv')
country_mapping = pd.read_csv('Country_Name_ISO3.csv')

# Display the initial number of rows of 'Country_Name_ISO3.csv' dataset
print(f"Initial number of rows: {country_mapping.shape[0]}")

# Remove rows where 'country_name' is empty or NaN in 'Country_Name_ISO3.csv' dataset
country_mapping_cleaned = country_mapping[country_mapping['country_name'].notna() & (country_mapping['country_name'].str.strip() != '')]

# Display the number of rows after cleaning of 'Country_Name_ISO3.csv' dataset
print(f"Number of rows after cleaning: {country_mapping_cleaned.shape[0]}")

# Ensure consistent formatting in the country_iso3 column
foreign_names['country_name'] = foreign_names['foreigncountry_cleaned'].str.strip()
country_mapping['country_name'] = country_mapping['country_name'].str.strip()

# Merge the sample with country mapping to include country names
foreign_names = foreign_names.merge(country_mapping, on='country_name', how='left')

# Function to clean firm names
def clean_name(name):
    if isinstance(name, str):  # Ensure the value is a string
        name = name.lower()
        name = name.replace('.', '')  # Remove periods
        name = ' '.join(name.split())  # Remove extra spaces
        return name.title()
    else:
        return ''  # Handle non-string values by returning an empty string

# Function to match and assign cleaned name, unique ID, and matching score
def assign_cleaned_name_id(df):
    cleaned_names = {}
    unique_id = 1
    
    # Function to find the best fuzzy match from a list of cleaned names
    def match_name(name, list_names, min_score=80):
        max_score = -1  # Initial score
        max_name = ""   # Initial best match
        for name2 in list_names:
            score = fuzz.ratio(name, name2)
            if score > min_score and score > max_score:
                max_name = name2
                max_score = score
        return max_name, max_score
    
    cleaned_name_list = []
    cleaned_id_list = []
    matching_score_list = []
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Cleaning and Matching Firms for {df['country_name'].iloc[0]}"):
        original_name = row['foreign']
        cleaned_name = clean_name(original_name)
        
        # Find the best match for the cleaned name
        best_match, best_score = match_name(cleaned_name, list(cleaned_names.keys()), min_score=80)
        
        if best_match:
            # Use the matched name as the cleaned name
            cleaned_name_list.append(best_match)
            cleaned_id_list.append(f"{cleaned_names[best_match]}_{df['country_iso3'].iloc[0]}")
            matching_score_list.append(best_score)
        else:
            # Use the current cleaned name and assign a new ID
            cleaned_names[cleaned_name] = unique_id
            cleaned_name_list.append(cleaned_name)
            cleaned_id_list.append(f"{unique_id}_{df['country_iso3'].iloc[0]}") 
            matching_score_list.append(0)  # No match found, so score is 0
            unique_id += 1
    
    # Use .loc[] to set the values
    df.loc[:, 'cleaned_name'] = cleaned_name_list
    df.loc[:, 'cleaned_ID'] = cleaned_id_list
    df.loc[:, 'matching_score'] = matching_score_list
    
    return df

# Group by country and apply the cleaning and matching function
cleaned_df_list = []
for country, group in tqdm(foreign_names.groupby('country_iso3'), desc="Processing by Country"):
    cleaned_group = assign_cleaned_name_id(group)
    cleaned_df_list.append(cleaned_group)

# Concatenate all the cleaned groups into one DataFrame
cleaned_df = pd.concat(cleaned_df_list, ignore_index=True)

# Select the specified columns
final_columns = ['foreign', 'foreigncountry_cleaned', 'country_iso3', 'shpmtyear', 'cleaned_name', 'cleaned_ID', 'matching_score']
cleaned_df = cleaned_df[final_columns]

# Save the cleaned dataset
cleaned_df.to_csv('Cleaned_ForeignNames_2019_2020.csv', index=False)

# Display a sample of the DataFrame
display(cleaned_df.head(20))

Initial number of rows: 285
Number of rows after cleaning: 284


Cleaning and Matching Firms for Andorra: 100%|██████████| 4/4 [00:00<00:00, 7466.50it/s]
Cleaning and Matching Firms for Netherlands Antilles: 100%|██████████| 135/135 [00:00<00:00, 15048.53it/s]
Cleaning and Matching Firms for United Arab Emirates: 100%|██████████| 7510/7510 [00:11<00:00, 640.80it/s]
Cleaning and Matching Firms for Bahamas: 100%|██████████| 47/47 [00:00<00:00, 24777.81it/s]
Cleaning and Matching Firms for Bosnia and Herzegovina: 100%|██████████| 58/58 [00:00<00:00, 28032.91it/s]
Cleaning and Matching Firms for Barbados: 100%|██████████| 61/61 [00:00<00:00, 26112.73it/s]
Cleaning and Matching Firms for Brunei Darussalam: 100%|██████████| 703/703 [00:00<00:00, 5144.64it/s]
Cleaning and Matching Firms for Botswana: 100%|██████████| 23/23 [00:00<00:00, 26590.13it/s]
Cleaning and Matching Firms for Central African Republic: 100%|██████████| 19/19 [00:00<00:00, 29330.80it/s]
Cleaning and Matching Firms for Denmark: 100%|██████████| 2054/2054 [00:01<00:00, 1917.71it/s]
Clean

,foreign,foreigncountry_cleaned,shpmtyear,country_name,country_iso3,cleaned_name,cleaned_ID,matching_score
0,Home & Nature Unicon N.V,Aruba,2019,Aruba,ABW,Home & Nature Unicon Nv,1_ABW,0
1,Gca Auto Sales N V,Aruba,2019,Aruba,ABW,Gca Auto Sales N V,2_ABW,0
2,Tricen Maintenance Services,Aruba,2019,Aruba,ABW,Tricen Maintenance Services,3_ABW,0
3,Gca Auto Sales N V,Aruba,2020,Aruba,ABW,Gca Auto Sales N V,2_ABW,100
4,Osyth Henriquez,Aruba,2019,Aruba,ABW,Osyth Henriquez,4_ABW,0
5,Landglass Distibutor Ind,Aruba,2020,Aruba,ABW,Landglass Distibutor Ind,5_ABW,0
6,Simple Beauty Trading N V,Aruba,2019,Aruba,ABW,Simple Beauty Trading N V,6_ABW,0
7,Home & Nature Unicon N.V,Aruba,2020,Aruba,ABW,Home & Nature Unicon Nv,1_ABW,100
8,R.E. Yrausquin And Sons (Aruba) N.V.,Aruba,2020,Aruba,ABW,Re Yrausquin And Sons (Aruba) Nv,7_ABW,0
9,Ultimate Automobiles N V,Aruba,2019,Aruba,ABW,Ultimate Automobiles N V,8_ABW,0


+ Divide the Data into Training and Test Sets and Apply Machine Learning: First, divide the cleaned data into training and test sets, typically using 70% for training and 30% for testing. Apply your initial cleaning algorithm to the training data, and then manually review and correct any errors in the cleaned names. Develop a machine learning model to predict the cleaned firm names (cleaned_name) based on the manually corrected training data. The variable to be predicted with the ML algorithm is the cleaned_name, as the goal is to standardize and accurately identify firm names despite inconsistencies in the original entries. Apply the trained model to the test data to see if it can generalize the cleaning rules and corrections. Finally, compare the accuracy of the cleaned names in the test data before and after applying the machine learning model by measuring the proportion of correctly predicted cleaned names. Comment on whether the accuracy has increased after applying the machine learning algorithm.

+ Include the output in a csv file that you will name outputfile_yourfirstname_1 (this file will include all the observations and all the fields in the ForeignNames_2019_2020.csv dataset + a variable called “cleaned_ID” (which is the ID you have given) + a variable called “cleaned_name” (the firm name you have given)).

+ Construct also a csv file that you will name outputfile_yourfirstname_1_changed that includes ONLY the firm names that have changed as a result of your cleaning + the original firm name that cleaned firm names changed to. In our example, this will be a dataset with all the “Teaboard” firms. Teaboard LTd, Teaboar Ld are the firm names that have changed as a result of the cleaning while Teaboard Limited is the original firm name that cleaned firm names changed to. Please do NOT include in this file those firms whose names have not changed).

In [7]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from IPython.display import display

# Now, divide the data into training and test sets
# Load the cleaned dataset
cleaned_df = pd.read_csv('Cleaned_ForeignNames_2019_2020.csv')

# Drop rows with NaN values in 'foreign' and 'cleaned_name'
cleaned_df = cleaned_df.dropna(subset=['foreign', 'cleaned_name'])

# Define features and labels
X = cleaned_df['foreign']  # Original names
y = cleaned_df['cleaned_name']  # Cleaned names

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Develop and train a machine learning model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Apply the model to the entire dataset to predict cleaned names
cleaned_df['cleaned_name_pred'] = model.predict(cleaned_df['foreign'])

# Save the output file with all observations and fields
cleaned_df.to_csv('outputfile_Jairo_1.csv', index=False)

# Construct a CSV file with only the firm names that have changed after applying the ML model
changed_names = cleaned_df[cleaned_df['cleaned_name'] != cleaned_df['cleaned_name_pred']]
changed_names.to_csv('outputfile_Jairo_1_changed.csv', index=False)

# Display samples of the output
display(cleaned_df.head())
display(changed_names.head())

: 

# Part 2

Use ForeignNames_2021.csv for the following tasks:

+ You did all the above cleaning work using a list of firm names until 2020. But the data on foreign names keeps getting updated, so now you need to expand the list of firm names to go up to 2021. You need to clean the names and assign the same IDs to firms that already exist prior to 2021 and new IDs to firms that appear just in 2021. For example, if Teaboard Ltd appears in 2021, it will get the same ID as before (2214, prefixed by the country 3-digit ISO codes). It is important to note that we again want all variations of Teaboard Ltd. to get the same unique ID just like before. 

+ The output from this task should be a file with all the original firm names, the variable “cleaned_name”, the variable “cleaned_ID”, a binary variable called “new” (taking value 1 if the firm name is new in 2021 and 0 otherwise). If the variable “new” takes a value 0, please include the name of the firm from the older data (i.e. the 2019-2020 data) in a variable called “old_name”. Obviously, both the old and new IDs should be the same. 

+ Include the output in a csv file that you will name outputfile_yourfirstname_2 

In [6]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm

# Set the working directory
os.chdir('/Users/jairogalvis/Downloads/PythonRStataTest2024/PythonTest/')

# Load the datasets
foreign_names_2021 = pd.read_csv('ForeignNames_2021.csv')
print(f"ForeignNames_2021 shape: {foreign_names_2021.shape}")
cleaned_df_2019_2020 = pd.read_csv('Cleaned_ForeignNames_2019_2020.csv')
print(f"Cleaned_ForeignNames_2019_2020 shape: {cleaned_df_2019_2020.shape}")
country_mapping = pd.read_csv('Country_Name_ISO3.csv')

# Clean 'Country_Name_ISO3.csv'
country_mapping_cleaned = country_mapping[country_mapping['country_name'].notna() & (country_mapping['country_name'].str.strip() != '')]
foreign_names_2021['foreigncountry_cleaned'] = foreign_names_2021['foreigncountry_cleaned'].str.strip()
country_mapping_cleaned['country_name'] = country_mapping_cleaned['country_name'].str.strip()

# Merge the 2021 data with country names
foreign_names_2021 = foreign_names_2021.merge(country_mapping_cleaned, left_on='foreigncountry_cleaned', right_on='country_name', how='left')

# Create a dictionary for cleaned IDs from 2019-2020 data
id_map = pd.Series(cleaned_df_2019_2020['cleaned_ID'].values, index=cleaned_df_2019_2020['cleaned_name']).to_dict()

# Function to clean firm names
def clean_name(name):
    if isinstance(name, str):
        name = name.lower()
        name = name.replace('.', '').replace(',', '')  # Remove periods and commas
        name = ' '.join(name.split())  # Remove extra spaces
        return name.title()
    return ''

# Clean and assign IDs to the 2021 data
def assign_id_and_name(df):
    cleaned_names = {}
    existing_ids = [int(id.split('_')[1]) for id in id_map.values() if id.split('_')[1].isdigit()]
    unique_id = max(existing_ids, default=0) + 1
    
    cleaned_name_list = []
    cleaned_id_list = []
    new_list = []
    old_name_list = []
    
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing Firms for {df['country_iso3'].iloc[0]}"):
        original_name = row['foreign']
        cleaned_name = clean_name(original_name)
        
        if cleaned_name in id_map:
            # Existing firm, use old ID
            cleaned_name_list.append(cleaned_name)
            cleaned_id_list.append(id_map[cleaned_name])
            new_list.append(0)
            old_name_list.append(cleaned_name)  # Keep the cleaned name as old name
        else:
            # New firm, assign a new ID
            cleaned_names[cleaned_name] = f"{df['country_iso3'].iloc[0]}_{unique_id}"
            cleaned_name_list.append(cleaned_name)
            cleaned_id_list.append(cleaned_names[cleaned_name])
            new_list.append(1)
            old_name_list.append('')  # No old name for new firms
            unique_id += 1
    
    df['cleaned_name'] = cleaned_name_list
    df['cleaned_ID'] = cleaned_id_list
    df['new'] = new_list
    df['old_name'] = old_name_list
    
    return df

# Combine the old and new datasets
combined_df = pd.concat([cleaned_df_2019_2020, foreign_names_2021], ignore_index=True)

# Process the combined dataset
processed_combined_df = assign_id_and_name(combined_df)

# Save the combined output to CSV
processed_combined_df.to_csv('Cleaned_ForeignNames_2019_2021.csv', index=False)

# Display a sample of the output
print(f"Whole dataset shape: {processed_combined_df.shape}")
display(processed_combined_df.head(20))


ForeignNames_2021 shape: (249656, 3)
Cleaned_ForeignNames_2019_2020 shape: (594032, 8)


/var/folders/bg/2pq2ytns0s7fgw4rcxpv0szw0000gn/T/ipykernel_93095/1113511913.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_mapping_cleaned['country_name'] = country_mapping_cleaned['country_name'].str.strip()
Processing Firms for ABW: 100%|██████████| 843688/843688 [00:19<00:00, 43972.55it/s]


Whole dataset shape: (843688, 10)


,foreign,foreigncountry_cleaned,shpmtyear,country_name,country_iso3,cleaned_name,cleaned_ID,matching_score,new,old_name
0,Home & Nature Unicon N.V,Aruba,2019,Aruba,ABW,Home & Nature Unicon Nv,1_ABW,0.0,0,Home & Nature Unicon Nv
1,Gca Auto Sales N V,Aruba,2019,Aruba,ABW,Gca Auto Sales N V,2_ABW,0.0,0,Gca Auto Sales N V
2,Tricen Maintenance Services,Aruba,2019,Aruba,ABW,Tricen Maintenance Services,3_ABW,0.0,0,Tricen Maintenance Services
3,Gca Auto Sales N V,Aruba,2020,Aruba,ABW,Gca Auto Sales N V,2_ABW,100.0,0,Gca Auto Sales N V
4,Osyth Henriquez,Aruba,2019,Aruba,ABW,Osyth Henriquez,4_ABW,0.0,0,Osyth Henriquez
5,Landglass Distibutor Ind,Aruba,2020,Aruba,ABW,Landglass Distibutor Ind,5_ABW,0.0,0,Landglass Distibutor Ind
6,Simple Beauty Trading N V,Aruba,2019,Aruba,ABW,Simple Beauty Trading N V,6_ABW,0.0,0,Simple Beauty Trading N V
7,Home & Nature Unicon N.V,Aruba,2020,Aruba,ABW,Home & Nature Unicon Nv,1_ABW,100.0,0,Home & Nature Unicon Nv
8,R.E. Yrausquin And Sons (Aruba) N.V.,Aruba,2020,Aruba,ABW,Re Yrausquin And Sons (Aruba) Nv,7_ABW,0.0,0,Re Yrausquin And Sons (Aruba) Nv
9,Ultimate Automobiles N V,Aruba,2019,Aruba,ABW,Ultimate Automobiles N V,8_ABW,0.0,0,Ultimate Automobiles N V


In [ ]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from IPython.display import display

# Now, divide the data into training and test sets

# Load the cleaned dataset
cleaned_df = pd.read_csv('Cleaned_ForeignNames_2019_2021.csv')

# Drop rows with NaN values in 'foreign' and 'cleaned_name'
cleaned_df = cleaned_df.dropna(subset=['foreign', 'cleaned_name'])

# Define features and labels
X = cleaned_df['foreign']  # Original names
y = cleaned_df['cleaned_name']  # Cleaned names

# Split the data into training (70%) and testing (30%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Develop and train a machine learning model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Apply the model to the entire dataset to predict cleaned names
cleaned_df['cleaned_name_pred'] = model.predict(cleaned_df['foreign'])

# Save the output file with all observations and fields
cleaned_df.to_csv('outputfile_Jairo_2.csv', index=False)

# Display samples of the output
display(cleaned_df.head())
display(changed_names.head())